# 布林拐头策略

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略

买入信号：波浪均线出现向上拐头（前两日均下降，今日上升）且当日收盘价在布林带中线之上。

买入仓位：根据海龟ATR仓位计算最优仓位。

卖出信号：波浪均线出现向下拐头（前两日均上升，今日下降）或收盘价向下突破布林带上界。

卖出仓位：一律平仓。

### 个股版本

In [2]:
class BW_s(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('direction_lag', 5),
        ('wave_period', 7),
        
        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):

        # Initialization
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Alias
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Indicators
        ## Bollinger
        self.bollinger = btind.BollingerBands(self.dataclose, 
                                              period=self.p.bollinger_per, 
                                              devfactor=self.p.bollinger_dev)

        ## Wave indicator
        self.sma = btind.SMA(self.dataclose, period=self.p.wave_period)

        ## ATR indicator
        self.atr_initial = self.broker.get_cash()
        self.tr = btind.Max((self.datahigh - self.datalow), 
                            abs(self.dataclose(-1) - self.datahigh),
                            abs(self.dataclose(-1) - self.datalow))
        self.atr = btind.SMA(self.tr, period=self.p.atr_period)

        # Signals
        self.bollinger_sell = bt.And(self.dataclose(0) < self.bollinger.top(0),
                                     self.dataclose(-1) > self.bollinger.top(-1))
        self.wave_buy = bt.And(self.sma(0) > self.sma(-1), self.sma(-1) < self.sma(-2), 
                               self.sma(-2) < self.sma(-3), self.sma(0) > self.bollinger.mid(0))
        self.wave_sell = bt.And(self.sma(0) < self.sma(-1), self.sma(-1) > self.sma(-2), 
                                self.sma(-2) > self.sma(-3))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, -order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None



    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        buy_s = self.wave_buy
        sell_s = self.bollinger_sell or self.wave_sell
        
        if buy_s and self.getposition(self.data).size == 0:
            # ATR position
            if self.p.using_atr:
                opt_pos = np.round(self.atr_initial * self.p.atr_percent / self.p.p_stake / self.atr[0])
                if opt_pos*self.p.p_stake > 0 and self.getposition(self.data).size < opt_pos*self.p.p_stake:
                    self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % 
                             (self.dataclose[0], opt_pos*self.p.p_stake-self.getposition(self.data).size,
                             self.getposition(self.data).size))
                    self.order = self.buy(size=opt_pos*self.p.p_stake-self.getposition(self.data).size)
            # Normal position
            else:
                cash_ava = self.broker.get_cash() / self.p.units
                lots = np.floor(cash_ava / (self.p.p_stake*self.dataclose[0]))*self.p.p_stake
                self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % 
                         (self.dataclose[0], lots, self.getposition(self.data).size))
                self.order = self.buy(size=lots)
        elif sell_s and self.getposition(self.data).size > 0:
            self.log('Sell CREATE(Close), Price: %.2f, Current Position: %i' % 
                     (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()

    
    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

## 回测
### 个股回测

In [4]:
stock_index = '000166.SZ'
startdate = dt.date(2020,12,31) - dt.timedelta(days=385)
endddate = dt.date(2020,12,31)


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(BW_s, printlog=True, direction_lag=20) 

    # Create stock Data Feed
    df = utilsJ.get_stock(stock_index, startdate, endddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=endddate)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 20000.00
2020-03-04: BUY CREATE, Price: 4.92, Lots: 4000, Current Position: 0
2020-03-05: BUY EXECUTED, Price:4.92, Lot:4000, Cash:320.
2020-03-09: Sell CREATE(Close), Price: 4.79, Current Position: 4000
2020-03-10: SELL EXECUTED, Price:4.79, Lot:4000, Cash:19480.
2020-03-10: OPERATION PROFIT, GROSS -520.00, NET -520.00
2020-06-19: BUY CREATE, Price: 4.60, Lots: 4200, Current Position: 0
2020-06-22: BUY EXECUTED, Price:4.60, Lot:4200, Cash:160.
2020-06-29: Sell CREATE(Close), Price: 4.81, Current Position: 4200
2020-06-30: SELL EXECUTED, Price:4.81, Lot:4200, Cash:20362.
2020-06-30: OPERATION PROFIT, GROSS 882.00, NET 882.00
2020-11-20: BUY CREATE, Price: 5.30, Lots: 3800, Current Position: 0
2020-11-23: BUY EXECUTED, Price:5.30, Lot:3800, Cash:222.
2020-11-24: Sell CREATE(Close), Price: 5.36, Current Position: 3800
2020-11-25: SELL EXECUTED, Price:5.36, Lot:3800, Cash:20590.
2020-11-25: OPERATION PROFIT, GROSS 228.00, NET 228.00
Final Portfolio Value: 20590.0

### 沪深300回测(调参)

In [ ]:
index_code = '399300.SZ'
startdate = dt.date(2020,12,31) - dt.timedelta(days=385)
enddate = dt.date(2020,12,31)

if __name__ == '__main__':

    index_list = utilsJ.get_index_components(index_code, startdate, enddate)
    profit_stk = dict()
    for stk in index_list:
        max_profit = (0,0.0,0)
        for i in range(3,7):
            for j in np.arange(1.5,3,0.1):
                # Create a cerebro entity
                cerebro = bt.Cerebro()

                # Add a strategy
                strats = cerebro.addstrategy(BW_s, printlog=False, direction_lag=i, bollinger_dev=float(j)) 

                # Create stock Data Feed
                df = utilsJ.get_stock(stk, startdate, enddate)
                data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)

                # Add the index Data Feed to Cerebo
                cerebro.adddata(data)

                # Set cash inside the strategy
                cerebro.broker = bt.brokers.BackBroker(coc=True)   
                cerebro.broker.setcash(300000)

                # Set commission
                #cerebro.broker.setcommission()

                # Record the starting conditions
                start_value = cerebro.broker.getvalue()
                #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

                # Run over everything
                cerebro.run()

                # Record the final result
                final_value = cerebro.broker.getvalue()
                #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
                
                print('%s, (%i, %.1f) Net Profit: %.2f%%' % (stk,i,j,(final_value - start_value) / start_value * 100))
                if max_profit[2] <  (final_value-start_value)/start_value*100:
                    max_profit = (i,j,(final_value-start_value)/start_value*100)
        profit_stk[stk] = (final_value - start_value) / start_value * 100